In [3]:
import math
import numpy as np
from numpy.random import shuffle, seed, choice
import nltk
from tqdm import tqdm
from collections import defaultdict

# First time you will need to download the corpus:
# Run the following and download the book collection

from nltk.corpus import brown
nltk.download('brown')



[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

## Bag of N-Grams

A BOW is a language modelling technique (also called a Term Frequency Vector) which creates a frequency distribution for a set of tokens -- or unigrams!
Extending this idea a bit, we can also create a Bag of N-Grams, which is a frequency distribution for
a set of N-grams for some N. If we divide the frequency by the number of N-grams, we have a probability distribution, such as we showed for the exciting text about John and Mary in Lecture 5.

For this homework, we are going to create such Bag of N-Gram models for N = 1, 2, 3, & 4, for the sentences
in `brown.sents()`.  We will evaluate them using a test set, and then in the second part of the
homework, we shall use them to generate sentences.


**Note 1:**  We do not want to do the same low-level transformations in this project
as we did in HW 02. We will keep the capitalization, punctuation, and
words in all their various forms.  There are some strange things in `brown.sents()`, such as double semicolons, and bad sentence segmentation, but we will assume
the processing of the texts into `brown.sents()` was consistent, and we will see what
our model makes of this data.

**Note 2:** Since `brown.sents()` contains punctuation marks as well as words, we shall use the term **tokens**
for the strings stored in the sentence lists.

### Part A: Randomize the list of sentences and split into training and testing sets

We will use  `brown.sents()` (a list of list of tokens) as the basis of our N-gram models.
The list `brown.words()` is simply the concatenation of all these lists of tokens.

We will shuffle the list into a random order, but using a seed value so that the order of the random
shuffle is the same each time.

1. Read about `numpy.random.seed` and `numpy.random.shuffle`.

2. Set the seed to `0` and shuffle the list: you can't shuffle `brown.sents()` and because `numpy.random.shuffle` modifies the list **in place**, to avoid reshuffling:
<ul>
    <li> Convert <b>brown.sents()</b> to a list and assign to a new variable <b>sentences</b> and then
    <li> <b>Copy sentences</b> to a new variable <b>shuffled_sentences</b> and then
    <li> Shuffle that list.
</ul>

In this way, you will have the original list, and a randomized list, but because of `seed(0)` it will be in the same order every time you run your code (and when we grade it).

3. Then split  `shuffled_sentences` into sets `training_sents` (first 99.9% of the sentences) and `testing_sents` (last 0.1%).

4. Print out the length of the training and testing sets.

5. Print out the first sentence in each of these sets.  

In 4 and 5, label the outputs so we know which is which. (Always make outputs easy
to understand!)

NOTE: The terms "training set" and "testing set" are very standard, even though we store these in lists (it is
possible that there are duplicate sentences).


In [4]:
# First, shuffle the set of sentences
seed(0)
# your code here
sentences = list(brown.sents())
shuffled_sentences = sentences.copy()
shuffle(shuffled_sentences)
split = int(0.999 * len(shuffled_sentences))

training_sents = shuffled_sentences[:split]
testing_sents = shuffled_sentences[split:]

# Print the length of the training and testing sets
print("Train set length:", len(training_sents))
print("Test set length:", len(testing_sents))

# Print the first sentence in each set
print("Train set 1st sentence:")
print(training_sents[0])

print("Test set 1st sentence:")
print(testing_sents[0])


Train set length: 57282
Test set length: 58
Train set 1st sentence:
['Muscle', 'weakness', 'did', 'not', 'improve', ',', 'and', 'the', 'patient', 'needed', 'first', 'a', 'cane', ',', 'then', 'crutches', '.']
Test set 1st sentence:
['It', 'is', 'at', 'least', 'as', 'important', 'as', 'the', 'more', 'dramatic', 'attempts', 'to', 'break', 'down', 'barriers', 'of', 'inequality', 'in', 'the', 'South', '.']


### Part B

Now, you must add the beginning `<s>` and ending `</s>` markers to each sentence in both the
training and testing lists.   Do not make any other changes
to the sentences -- you will see that punctuation has been left in, such as periods at the end
of sentences. Again, we will see what our models make of this data set.

Print out the first sentence in each of the training and testing sets to check that all is well.

In [5]:
# put `<s>` at beginning and `</s>` at end of all sentences.

def bracket_sentence(sent):
  return ['<s>'] + sent + ['</s>']



# your code here
training_sents = [bracket_sentence(sent) for sent in training_sents]
testing_sents = [bracket_sentence(sent) for sent in testing_sents]

print("Train set 1st sentence:")
print(training_sents[0])

print("Test set 1st sentence:")
print(testing_sents[0])


Train set 1st sentence:
['<s>', 'Muscle', 'weakness', 'did', 'not', 'improve', ',', 'and', 'the', 'patient', 'needed', 'first', 'a', 'cane', ',', 'then', 'crutches', '.', '</s>']
Test set 1st sentence:
['<s>', 'It', 'is', 'at', 'least', 'as', 'important', 'as', 'the', 'more', 'dramatic', 'attempts', 'to', 'break', 'down', 'barriers', 'of', 'inequality', 'in', 'the', 'South', '.', '</s>']


###  Part C

Complete the following template for a function to extract N-grams from one sentence, and test
it for N = 1,2,3,4 for the first sentences in the training set.

In [6]:
# Return a list of the N-grams for all sentences s

# Store all N-grams as tuples, so that a unigram is (w,), a bigram is (w1,w2), etc.

def get_Ngrams_for_sentence(N,s):
    s = tuple(s)
    grams = []
    for i in range(len(s) - N + 1):
        gram = s[i:i + N]
        grams.append(gram)
    return grams


# your code here
for i in range(1,5):
  ngram = get_Ngrams_for_sentence(i,training_sents[0])
  print(ngram,"\n")


[('<s>',), ('Muscle',), ('weakness',), ('did',), ('not',), ('improve',), (',',), ('and',), ('the',), ('patient',), ('needed',), ('first',), ('a',), ('cane',), (',',), ('then',), ('crutches',), ('.',), ('</s>',)] 

[('<s>', 'Muscle'), ('Muscle', 'weakness'), ('weakness', 'did'), ('did', 'not'), ('not', 'improve'), ('improve', ','), (',', 'and'), ('and', 'the'), ('the', 'patient'), ('patient', 'needed'), ('needed', 'first'), ('first', 'a'), ('a', 'cane'), ('cane', ','), (',', 'then'), ('then', 'crutches'), ('crutches', '.'), ('.', '</s>')] 

[('<s>', 'Muscle', 'weakness'), ('Muscle', 'weakness', 'did'), ('weakness', 'did', 'not'), ('did', 'not', 'improve'), ('not', 'improve', ','), ('improve', ',', 'and'), (',', 'and', 'the'), ('and', 'the', 'patient'), ('the', 'patient', 'needed'), ('patient', 'needed', 'first'), ('needed', 'first', 'a'), ('first', 'a', 'cane'), ('a', 'cane', ','), ('cane', ',', 'then'), (',', 'then', 'crutches'), ('then', 'crutches', '.'), ('crutches', '.', '</s>')] 



###  Part D

Now create lists of N-grams for all the sentences in your training set (NOT the testing set).
Complete the following template to assign these to the given list.

Print out the number of N-grams, and the first 5 N-grams in each list for N = 1, 2, 3, 4.

Note that this number is the number of occurrences of N-grams, which may not be unique in the list.

In [7]:
Ngrams = [None]*5    # first slot is empty, then Ngram[1] will hold unigrams, Ngram[2] will hold bigrams, etc.

# your code here
for N in range(1, 5):
    Ngrams[N] = []

for N in range(1, 5):
  for sentence in training_sents:
    ngrams = get_Ngrams_for_sentence(N, sentence)
    Ngrams[N].extend(ngrams)

for N in range(1, 5):
  print(f"There are {len(Ngrams[N])} N-grams in Ngrams[{N}] and the first 5 are:\n {Ngrams[N][:5]}\n")

There are 1274667 N-grams in Ngrams[1] and the first 5 are:
 [('<s>',), ('Muscle',), ('weakness',), ('did',), ('not',)]

There are 1217385 N-grams in Ngrams[2] and the first 5 are:
 [('<s>', 'Muscle'), ('Muscle', 'weakness'), ('weakness', 'did'), ('did', 'not'), ('not', 'improve')]

There are 1160103 N-grams in Ngrams[3] and the first 5 are:
 [('<s>', 'Muscle', 'weakness'), ('Muscle', 'weakness', 'did'), ('weakness', 'did', 'not'), ('did', 'not', 'improve'), ('not', 'improve', ',')]

There are 1102821 N-grams in Ngrams[4] and the first 5 are:
 [('<s>', 'Muscle', 'weakness', 'did'), ('Muscle', 'weakness', 'did', 'not'), ('weakness', 'did', 'not', 'improve'), ('did', 'not', 'improve', ','), ('not', 'improve', ',', 'and')]



### Part E

We will now create a probability distribution for each of the Ngram collections. Note carefully
that you must divide the frequency of each N-gram by the number of occurrences of N-grams, not
the number of unique N-grams.

Note that we have set the probability of the unigram `<s>` to 1.0. This is because we are
interested in calculating the probability of sentences, which *always* begin with the token `'<s>'`.

Complete the following template and then

1. Print out the total number of N-grams in each dictionary (they should be a bit smaller than the totals in the last part - why?).
2. Test your code by printing out the probability of the following Ngrams to 8 digits of precision:

        ('to',)           
        ('to','the')            
        ('to','the','house')              
        ('to','the','house','.')

In [8]:
# Create a defaultdict with the frequency distribution for the training set for a given N.

# your code here

def get_Ngram_distribution(N,Ngrams):
  ngram_freq = defaultdict(lambda: 0)
  ngram_dist = defaultdict(lambda: 0)
  for ngram in Ngrams:
      ngram_freq[ngram] += 1

  for ngram in ngram_freq:
      ngram_dist[ngram] = ngram_freq[ngram]/len(Ngrams)

  return ngram_dist

# now create for N = 1,2,3,4

Ngram_distribution = [None]*5

# your code here
for N in range(1, 5):
    Ngram_distribution[N] = get_Ngram_distribution(N, Ngrams[N])

#The counts are a little less because we are getting a count for each Ngram rather than the Ngrams itself
for N in range(1, 5):
    print(f"Length of Ngram_distribution[{N}]: {len(Ngram_distribution[N])}")

# Since all setences will start with <s>, its probability should be 1.0
Ngram_distribution[1][('<s>',)] = 1.0

# tests

print(f"\nThe Probability of ('<s>',) is {np.around(Ngram_distribution[1][('<s>',)],8)}.")
print(f"\nThe Probability of ('to',) is {np.around(Ngram_distribution[1][('to',)],8)}.")
print(f"\nThe Probability of ('to','the') is {np.around(Ngram_distribution[2][('to','the')],8)}.")
print(f"\nThe Probability of ('to','the','house') is {np.around(Ngram_distribution[3][('to','the','house')],8)}.")
print(f"\nThe Probability of ('to','the','house','.')  is {np.around(Ngram_distribution[4][('to','the','house','.') ],8)}.")

Length of Ngram_distribution[1]: 56029
Length of Ngram_distribution[2]: 454055
Length of Ngram_distribution[3]: 877117
Length of Ngram_distribution[4]: 1029391

The Probability of ('<s>',) is 1.0.

The Probability of ('to',) is 0.02017703.

The Probability of ('to','the') is 0.00281341.

The Probability of ('to','the','house') is 9.48e-06.

The Probability of ('to','the','house','.')  is 2.72e-06.


## Probability and Perplexity

Now we will calculate the probability and the perplexity of sequences
of tokens, using the principle of "Stupid Backoff" as explained
in the paper:

https://aclanthology.org/D07-1090.pdf

and explicated in this StackOverflow post:

https://stackoverflow.com/questions/16383194/stupid-backoff-implementation-clarification

Before describing "Stupid Backoff," let us consider the naive way to
calculate the probability of a sequence of tokens which starts with `<s>`.

#### A simple and naive way to calculate probabilities of sequences of tokens

Suppose we have a quadrigram model (N = 4), we have a sequence of tokens

$$w_1, w_2, \cdots, w_n,$$   

Assume we have calculated all the N-gram
probabilities in  `Ngram_distribution[N]` for N = 1,2,3,4.  


We will calculate the probability of each successive token $w_i$ in as much left context as we have, up to 3 (the
last token in the 4-gram being $w_i$).


$$\begin{aligned}
        p_1 &= \text{Ngram_distribution[1][($w_1$,)]}    \\   
        p_2 &=   \text{Ngram_distribution[2][$( w_1,w_2)$]}\ / \ \text{Ngram_distribution[1][$( w_1,)$]}    \\              p_3 &=   \text{Ngram_distribution[3][$( w_1,w_2,w_3)$]}\ / \  \text{Ngram_distribution[2][$(\text{'<s>'}, w_1,w_2)$]}   \\      
        p_4 &=   \text{Ngram_distribution[4][$( w_1,w_2,w_3,w_4)$]}\ / \  \text{Ngram_distribution[3][$(\text{'<s>'}, w_1,w_2,w_3)$]}   \\        
            &\ldots                             \\
        p_i &=   \text{Ngram_distribution[4][$(w_{i-3}, w_{i-2}, w_{i-1}, w_i)$]}\ /\  \text{Ngram_distribution[3][$(w_{i-3}, w_{i-2}, w_{i-1})$]}   \\        
                    &\ldots                            \\
        p_n &=  \text{Ngram_distribution[4][$(w_{n-3}, w_{n-2}, w_{n-1}, w_n)$]}\ /\  \text{Ngram_distribution[3][$(w_{n-3}, w_{n-2}, w_{n-1})$]}     \\        
  \end{aligned}$$
  
Note that if $w_1$ is `<s>`, then $p_1=1.0$.
    
Finally, let

$$P(\text{'<s>'}, w_1, w_2, \cdots, w_n)\ =\ p_1\ast p_2\ast\cdots\ast p_n.$$

One messy detail is dealing with the possibility of 0 counts (if that is possible); thus,
if the numerator in the above expressions is 0, then the entire product is 0 (you want to avoid
a divide by 0 error in the denominator).

#### What could possibly go wrong?

Well, if our sentence is from our training set, nothing!  All the probabilities will
have been calculated for all the possible N-grams.

However, when we have a separate
training set, we have to account for the fact that **some N-grams (and even
some tokens) may occur in the testing set which do not occur in the training set,
and so their probability will be 0.**  However, we want to make the best estimate
of the probability we can!

There are various solutions, which we discussed in lectures 5 and 6, but the simplest
(and very effective for large data sets) is "Stupid Backoff," recursively defined as follows
for bigrams, trigrams, and quadrigrams, and **using the probability calculations shown above.**

    PN_stupid_backoff(w1) = p1   as defined above           # if this is not 0, else:
                          = (frequency of w1 in whole corpus / number of tokens in whole corpus)
              
    PN_stupid_backoff(w1, w2) = p2   as defined above         # if this is not 0, else:
                              = 0.4 * P_stupid_backoff(w2)    # recursive, use previous definition

    PN_stupid_backoff(w1, w2, w3) = p3   as defined above           # if this is not 0, else:
                                  = 0.4 * P_stupid_backoff(w2, w3)  # recursive, use definition above

                       
    PN_stupid_backoff(w1, w2, w3, w4) = p4  as defined above                 # if this is not 0, else:
                                      = 0.4 * P_stupid_backoff(w2, w3, w4)   # recursive, previous definition

This accounts for how to backoff when trying to find the probability of some $w_i$ in a left context
of 1, 2, or 3 tokens.

Then we use these calculations instead of $p_1$, $p_2$ as in the previous algorithm, for trigrams
it would be:

     P_stupid_backoff(w1, w2, w3, w4, ..., wn) =   PN_stupid_backoff(w1)
                                                 * PN_stupid_backoff(w1, w2)
                                                 * PN_stupid_backoff(w1, w2, w3)
                                                 * PN_stupid_backoff(w2, w3, w4)
                                                 ...
                                                 * PN_stupid_backoff(w(n-2), w(n-1), wn)

The "discount factor"
0.4 was proposed by the originators of the method, and seems to work well in practice.

This calculation is unnecessary in generative models, since then we will train on the entire
corpus, and only use available N-grams to produce sentences.

## Problem 2

Now we will calculate the probability of a sequence of tokens. We will warm up by considering
the simple case, and then consider the more complex case, where
"stupid backoff" will be used.

### Part A

For this part, complete the following template to create a function which will
calculate the probability of a sequence of tokens.

**Since you may want to use this in the stupid backoff version, you should make sure that if the
numerator in the conditional probability calculation is 0, immediately return 0,
so that there is no possibility of divide by 0 in the denominator.**

Tests are provided following the cell in which you will write your code.

In [9]:
# Probability of a list of tokens using N-grams
# W a list of tokens

# Calculate the probability of the last token in W, as we did in the calculation of p1, p2, etc. above
# check numerator: if it is 0, return 0 immediately and do not do the division (to avoid possible
# division by 0)

# N == len(W)

def PN(N,W):
    if Ngram_distribution[N].get(W, 0) == 0:
        return 0

    numerator = Ngram_distribution[N].get(W, 0)
    if Ngram_distribution[N - 1] == None: # If N is 1 we do not have Ngram_distribution[0], so return numerator
        return Ngram_distribution[N].get(W, 0)
    else:
        return numerator / Ngram_distribution[N - 1].get(W[:-1], 0)

# Now calculate for a whole sequence: use PN(..) for bigram, trigram, etc. up
# model, then slide PN(...) across the sequence, as shown above.
def P(N,W):
    probability = 1.0
    step = N - 2          # Calculates the number of steps required for the expansion
    gram = 2
    #Expansion phase
    for j in range(step):
      p = PN(gram, W[:gram])
      gram+=1
      probability *= p
    #Sliding phase
    for i in range(N - 1, len(W)):
      p = PN(N, W[i - N + 1:i + 1])
      probability *= p
    return probability


The following are tests to make sure your code is working properly. The values printed
should be the same.

In [10]:
#  Sentence:  He frowned.
#  Using a bigram model

a = Ngram_distribution[2][('<s>','He')]
b = Ngram_distribution[2][('He','frowned')] / Ngram_distribution[1][('He',)]
c = Ngram_distribution[2][('frowned','.')] / Ngram_distribution[1][('frowned',)]
d = Ngram_distribution[2][('.','</s>')] / Ngram_distribution[1][('.',)]


print('a=', a)
print('  ', PN(2,('<s>','He')))
print('b =', b)
print('  ', PN(2,('He','frowned')))
print('c=', c)
print('  ', PN(2,('frowned','.')))
print('d=', d)
print('  ', PN(2,('.','</s>')))

print('\na*b*c*d:  ',a*b*c*d)
print('P(2,...): ', P(2,('<s>','He','frowned','.','</s>')))

a= 0.002346012148991486
   0.002346012148991486
b = 0.000351478118528877
   0.000351478118528877
c= 0.39264499316157175
   0.39264499316157175
d= 1.0470533150975245
   1.0470533150975245

a*b*c*d:   3.389982137368031e-07
P(2,...):  3.389982137368031e-07


In [11]:
#  Sentence:  He frowned.
#  Using a trigram model

a = Ngram_distribution[2][('<s>','He')]
b = Ngram_distribution[3][('<s>','He','frowned')] / Ngram_distribution[2][('<s>','He',)]
c = Ngram_distribution[3][('He','frowned','.')] / Ngram_distribution[2][('He','frowned',)]
d = Ngram_distribution[3][('frowned','.','</s>')] / Ngram_distribution[2][('frowned','.',)]

print('a*b*c*d:  ',a*b*c*d)
print('P(3,...): ', P(3,('<s>','He','frowned','.','</s>')))

a*b*c*d:   9.492186072583041e-07
P(3,...):  9.492186072583041e-07


In [12]:
#  Sentence:  He frowned.
#  Using a quadrigram model

a = Ngram_distribution[2][('<s>','He')]
b = Ngram_distribution[3][('<s>','He','frowned')] / Ngram_distribution[2][('<s>','He',)]
c = Ngram_distribution[4][('<s>','He','frowned','.')] / Ngram_distribution[3][('<s>','He','frowned',)]
d = Ngram_distribution[4][('He','frowned','.','</s>')] / Ngram_distribution[3][('He','frowned','.',)]

print('a*b*c*d:  ',a*b*c*d)
print('P(4,...): ', P(4,('<s>','He','frowned','.','</s>')))

a*b*c*d:   9.538640808692934e-07
P(4,...):  9.538640808692934e-07


In [13]:
# Sentence:  I love NLP
# using trigrams

# This shows that you should test the numerator to see if it is 0, because then
# you can return 0 immediately, and not have the possibility of division by 0,
# which would occur in the cases d below (c is 0 but would not cause division by 0)

a  = Ngram_distribution[2][('<s>','I')]
b  = Ngram_distribution[3][('<s>','I','love')] / Ngram_distribution[2][('<s>','I',)]
c  = Ngram_distribution[3][('I','love','NLP')] / Ngram_distribution[2][('I','love')]
d1 = Ngram_distribution[3][('love','NLP','</s>')]
d2 = Ngram_distribution[2][('love','NLP')]

print('a=',a,'\nb=',b,'\nc=',c,'\nd1=',d1,'\nd2=',d2,'\n')

print('a*b*d*d1:  ',a*b*c*d1)
print('P(3,...): ', P(3,('<s>','I','love','NLP','</s>')))

a= 0.001128648701930778 
b= 0.0015274769289326804 
c= 0.0 
d1= 0 
d2= 0 

a*b*d*d1:   0.0
P(3,...):  0.0


### Part B

Now we will develop the probability for a sequence with the possibility that some N-grams, or
even some tokens, are not in the training set. We will use the idea of "stupid backoff" explained
in lecture.

Complete the following template and verify that it passes all the tests.


In [14]:
# Probability with stupid backoff
# same as previous, but have to use recursive (or iterative) method instead of
# calling Ngram_distribution directly

# W a list of tokens
import re

sentences = list(brown.words())
num_all_tokens = len(brown.words())

# This returns backed-off probability for single N-gram
# len(W) must be N, this will try whole N-gram, then last N-1 tokens, then N-2, etc. down to 1 token.

# Assumes W is a tuple

# calculate for a particular length N-gram
# must have N == len(W)

def PN_with_stupid_backoff(N,W):
    if N == 1:
      if Ngram_distribution[N].get(W, 0) == 0:
        count = sentences.count(W[0])
        return count / num_all_tokens
    else:
      if Ngram_distribution[N].get(W, 0) == 0:
        backoff_prob = 0.4 * PN_with_stupid_backoff(N - 1, W[1:])
        return backoff_prob

    numerator = Ngram_distribution[N].get(W, 0)
    if Ngram_distribution[N - 1] == None:
        return Ngram_distribution[N].get(W, 0)
    else:
        return numerator / Ngram_distribution[N - 1].get(W[:-1], 0)



# Note that for training set, this will be same as P(N,W) since all probabilities are non-zero

# Really the same as P(N,W) except using the previous function, and no need to check for 0,
# since it never returns 0 for a sequence using words from the corpus



def P_stupid_backoff(N,W):
    probability = 1.0
    step = N - 2
    gram = 2
    #Expansion phase
    for j in range(step):
      p = PN_with_stupid_backoff(gram, W[:gram])
      gram+=1
      probability *= p
    #Sliding phase
    for i in range(N - 1, len(W)):
      p = PN_with_stupid_backoff(N, W[i - N + 1:i + 1])
      probability *= p
    return probability



The following are tests to make sure your code is working properly. The values printed
should be the same.

In [15]:
# 'grandstand' is in testing set but not in the training set
# This uses bigrams

P(2,('<s>','where','is','the','grandstand'))

0.0

In [16]:
a = PN(2,('<s>','where'))
b = PN(2,('where','is'))
c = PN(2,('is','the'))
d2 = PN(2,('the','grandstand'))     # this is 0, so use less context
d1 = PN(1,('grandstand'))           # this is 0, so use 0.4*d instead of d2
d  = list(brown.words()).count('grandstand') / len(brown.words())

print('a =',a,'\nb =',b,'\nc =',c,'\nd2=',d2,'\nd1=',d1,'\nd =',d,'\n')
print(a*b*c*(0.4*d))
print(P_stupid_backoff(2,('<s>','where','is','the','grandstand')))

a = 1.6428656505542618e-06 
b = 0.006166391726133832 
c = 0.08182229363508187 
d2= 0 
d1= 0 
d = 8.611840246918683e-07 

2.855359302895301e-16
2.855359302895301e-16


In [17]:
# 'grandstand' is in testing set but not in the training set
# This uses trigrams

P(3,('<s>','where','is','the','grandstand'))

0.0

In [18]:
a  = PN(2,('<s>','where'))               # <= this works
b3 = PN(3,('<s>','where','is'))         # this is 0, so try less context
b2 = PN(2,('where','is'))               # <= this works
c  = PN(3,('where','is','the'))          # <= this works

d3 = PN(3,('is','the','grandstand'))    # this is 0, so try less context
d2 = PN(2,('the','grandstand'))         # this is 0, so try less context
d1 = PN(1,('grandstand'))               # this is 0, so use probability in whole corpus
d  = list(brown.words()).count('grandstand') / len(brown.words())     # <= this works

print('a =',a,'\nb3=',b3,'\nb2=',b2,'\nc =',c,'\nd3=',d3,'\nd2=',d2,'\nd1=',d1,'\nd =',d,'\n')

# every time we try less context, must multiply by 0.4, so we
# use 0.4*b2 instead of b3 and 0.4*0.4*e instead of d3
print(a*(0.4*b2)*c*(0.4*0.4*d))
print(P_stupid_backoff(3,('<s>','where','is','the','grandstand')))

a = 1.6428656505542618e-06 
b3= 0 
b2= 0.006166391726133832 
c = 0.4197506600707006 
d3= 0 
d2= 0 
d1= 0 
d = 8.611840246918683e-07 

2.3436917228933544e-16
2.3436917228933544e-16


In [19]:
a  = PN(2,('<s>','The'))                   #   <= this works

b3 = PN(3,('<s>','The','grandstand'))      # this is 0, so try less context (smaller N)
b2 = PN(2,('The','grandstand'))            # this is 0, so try less context
b1 = PN(1,('grandstand',))                 # this is 0, so have to use frequency in whole corpus
b  = list(brown.words()).count('grandstand') / len(brown.words())    #   <= this works

c3 = PN(3,('The','grandstand','fell'))     # this is 0, so try less context
c2 = PN(2,('grandstand','fell'))           # this is 0, so try less context
c1 = PN(1,('fell',))                       # ok, this works

d3 = PN(3,('grandstand','fell','down'))   # this is 0, so try less context
d2 = PN(2,('fell','down'))                #   <= this works

e3 = PN(3,('fell','down','</s>'))         # this is 0, so try less context
e2 = PN(2,('down','</s>'))                #  <= this works

# every time we tried a smaller N, we have to multiply by 0.4
# so we use a, then 0.4*0.4*b0, then 0.4*0.4*c1, then 0.4*d2, then 0.4*e2e.

print('a =',a,'\nb3=',b3,'\nb2=',b2,'\nb1=',b1,'\nb =',b)
print('c3=',c3,'\nc2=',c2,'\nc1=',c1,'\nd3=',d3,'\nd2=',d2,'\ne3=',e3,'\ne2=',e2,'\n')

print(a * (0.4*0.4*b) * (0.4*0.4*c1) * (0.4*d2) * (0.4*e2) )
print(P_stupid_backoff(3,('<s>','The','grandstand','fell','down','</s>')))

a = 0.005372992110137713 
b3= 0 
b2= 0 
b1= 0 
b = 8.611840246918683e-07
c3= 0 
c2= 0 
c1= 7.21757133431712e-05 
d3= 0 
d2= 0.01138101429453831 
e3= 0 
e2= 0.0011817757506744071 

1.839836556015632e-20
1.839836556015632e-20


### Part C

Now we will implement the notion of *perplexity* as explained in lecture. Refer to
the formula presented there to complete the following template, and verify
that it passes all the tests.

In [20]:
# Perplexity

# We assume that W starts with <s>, may not end with </s>

def PP(N,W):
    probability = P_stupid_backoff(N,W)
    n = (len(W) - 1)
    perplexity = probability ** (-1/n)
    return perplexity



If we know that no probabilities can be 0,  then P is same as P_stupid_backoff

In [21]:
PP(2,('<s>','The'))

186.11603730316466

In [22]:
P(2,('<s>','The'))**(-1/1)

186.11603730316466

In [23]:
P_stupid_backoff(2,('<s>','The'))**(-1/1)

186.11603730316466

In [24]:
PP(2,('<s>','The','man','went'))

308.91157551766736

In [25]:
P(2,('<s>','The','man','went'))**(-1/3)

308.91157551766736

In [26]:
P_stupid_backoff(2,('<s>','The','man','went'))**(-1/3)

308.91157551766736

In [27]:
PP(2,('<s>','The','man','went','to','the', 'house','.','</s>'))

35.03849995731908

In [28]:
P(2,('<s>','The','man','went','to','the', 'house','.','</s>'))**(-1/8)

35.03849995731908

In [29]:
P_stupid_backoff(2,('<s>','The','man','went','to','the', 'house','.','</s>'))**(-1/8)

35.03849995731908

When probabilities may be 0, we must use stupid backoff

In [30]:
PP(2,('<s>','where','is','the','grandstand'))

7692.8065013245405

In [31]:
P_stupid_backoff(2,('<s>','where','is','the','grandstand'))**(-1/4)

7692.8065013245405

In [32]:
PP(3,('<s>','where','is','the','grandstand'))

8082.112259400884

In [33]:
P_stupid_backoff(3,('<s>','where','is','the','grandstand'))**(-1/4)

8082.112259400884

In [34]:
PP(3,('<s>','The','grandstand','fell','down','</s>'))

8852.055970670379

In [35]:
P_stupid_backoff(3,('<s>','The','grandstand','fell','down','</s>'))**(-1/5)

8852.055970670379

In [36]:
PP(4,('<s>','The','grandstand','fell','down','</s>'))

15339.392368477242

In [37]:
P_stupid_backoff(4,('<s>','The','grandstand','fell','down','</s>'))**(-1/5)

15339.392368477242

### Part D

Print out the first ten sentences in the training set, with their perplexities to 2 decimal places, using trigrams.  Then do the
same for the testing set.

Print out the text of the sentences in a readable form, e.g., for a sentence `w`, print it out using

    ' '.join(w[1:-1)
    
    
Notice the perplexities of the training set are generally smaller than the testing set!

In [38]:
print("Training Set Perplexities:")
for sentence in training_sents[:10]:
    perplexity = PP(3, tuple(sentence))
    sent = ' '.join(sentence[1:-1])
    print(f"{perplexity:.2f}\t {sent}")

Training Set Perplexities:
10.76	 Muscle weakness did not improve , and the patient needed first a cane , then crutches .
10.74	 He replaced the flashlight where it had been stowed , got into his own car and backed it out of the garage .
8.42	 When he had given the call a few moments thought , he went into the kitchen to ask Mrs. Yamata to prepare tea and sushi for the visitors , using the formal English china and the silver tea service which had been donated to the mission , then he went outside to inspect the grounds .
10.04	 -- On the basis of a differentiability assumption in function space , it is possible to prove that , for materials having the property that the stress is given by a functional of the history of the deformation gradients , the classical theory of infinitesimal viscoelasticity is valid when the deformation has been infinitesimal for all times in the past .
4.88	 She said sharks have no bones and shrimp swam backward .
9.62	 T. V. Barker , who developed the classif

In [39]:
print("Testing Set Perplexities:")
for sentence in testing_sents[:10]:
    perplexity = PP(3, tuple(sentence))
    sent = ' '.join(sentence[1:-1])
    print(f"{perplexity:.2f}\t {sent}")

Testing Set Perplexities:
176.79	 It is at least as important as the more dramatic attempts to break down barriers of inequality in the South .
1306.74	 the car's far windshield panel turned into a silver web with a dark hole in the center .
69.22	 `` I was just thinking how things have changed .
1071.75	 She smiled , and the teeth gleamed in her beautifully modeled olive face .
438.24	 `` There isn't a chance of Myra's letting anything like that happen .
174.63	 On the other hand , many a pastor is so absorbed in ministering to the intimate , personal needs of individuals in his congregation that he does little or nothing to lead them into a sense of social responsibility and world mission .
923.70	 We live down by the Base commissary .
484.32	 For example , the BBB has reported it was receiving four times as many inquiries about quack devices and 10 times as many complaints compared with two years ago .
204.82	 As a result , life had become a kind of continuous make-ready .
210.44	 S

### Part E

Finally, we will find the perplexities of the the testing set with bigrams, trigrams, and quadrigrams.
Complete the following template to verify that your results are consistent with the test results.

In [40]:
# Find all the probabilities of the sentences in the testing set, multiply them,
# and take the $K^{th}$ root, where K is the number of tokens, excluding the <s> tokens.
# So, K = (sum of length of sentences) - (# of sentences)

# We need to take the product of many small probabilities, so use math.log and math.exp to avoid
# underflow.

# Print out the perplexity as an integer.

import math

# your code here

def calculate_perplexity(N, testing_sents):
        total_log_prob = 0.0
        K = sum(len(sentence) - 1 for sentence in testing_sents)

        for sentence in testing_sents:
            sentence_tokens = tuple(sentence)
            p = P_stupid_backoff(N, sentence_tokens)
            total_log_prob += math.log(p)


        perplexity = math.exp(-total_log_prob / K)
        return int(perplexity)

# Calculate perplexities for bigrams, trigrams, and quadrigrams
print("Perplexity for Bigrams:", calculate_perplexity(2, testing_sents))
print("Perplexity for Trigrams:", calculate_perplexity(3, testing_sents))
print("Perplexity for Quadrigrams:", calculate_perplexity(4, testing_sents))


Perplexity for Bigrams: 387
Perplexity for Trigrams: 496
Perplexity for Quadrigrams: 957


## Problem 3: Generative N-Gram Model


Now we will consider how to generate sentences using our N-gram model.


The idea is fairly simple.  Suppose we have model using N=4 (quadrigrams -- the algorithm for bigrams and trigrams
is analogous):

1. To get $w_1$, choose a bigram $(\text{"<s>"}, w_1)$ randomly according the probability distribution stored in

$$\text{Ngram_distribution[2][ $(\text{"<s>"}, w_1)$ ]}.$$

2. To get $w_2$, choose a bigram $(\text{"<s>"}, w_1, w_2)$ randomly according the probability distribution calculated as

$$\text{Ngram_distribution[3][ $(\text{"<s>"},w_1,w_2)$ ]}\ /\ \text{Ngram_distribution[2][ $(\text{"<s>"},w_1)$ ]}.$$

3. To get $w_3$, choose a trigram $(\text{"<s>"}, w_1, w_2, w_3)$ randomly according the probability distribution calculated as

$$\text{Ngram_distribution[4][ $(\text{"<s>"},w_1,w_2, w_3)$ ]}\ /\ \text{Ngram_distribution[3][ $(\text{"<s>"},w_1,w_2)$ ]}.$$

4. Thereafter, for a sequence $(\text{"<s>"}, w_1, w_2, \ldots, w_{i-2}, w_{i-1})$, to get $w_i$, choose a
quadrigram $(w_{i-3}, w_{i-2}, w_{i-1}, w_i)$ randomly according the probability distribution stored in

$$\text{Ngram_distribution[4][ $(w_{i-3}, w_{i-2}, w_{i-1},w_i)$ ]}\ /\ \text{Ngram_distribution[3][ $(w_{i-3}, w_{i-2},w_{i-1})$ ]}.$$

5. When we generate the end of sentence marker `<\s>` we stop.

The problem is that this is difficult if we simply use the formulae given above: what we need is
a separate probability distribution for each prefix.

### Part A

The first step, under the assumption that we are working with N-grams for N = 1,2,3, or 4, is to build a data structure that can sample from the distribution of next tokens
given an (N-1)-gram of left context.

The best choice here is a nested default dictionary for N-grams for N = 2,3,4, the outer dictionary containing
keys consisting of the first N-1 tokens (we'll call this the *prefix*), with the value being an inner dictionary holding a probability distribution for the last token (we'll call this *wn*).

For this problem, you need to redo the construction of the list of N-grams and the distributions for
each N, using the *entire* set of sentences, not just the testing set you used for the previous problems.
You can easily do this by copying and pasting code from above.


In [41]:
All_Ngrams = [None]*5    # first slot is empty, then Ngram[1] will hold unigrams,
                         # Ngram[2] will hold bigrams, etc.

# your code here
sentences = training_sents + testing_sents
for N in range(1, 5):
   All_Ngrams[N] = []

for N in range(1, 5):
  for sentence in sentences:
    ngrams = get_Ngrams_for_sentence(N, sentence)
    All_Ngrams[N].extend(ngrams)

# now create for N = 1,2,3,4

All_Ngram_distribution = [None]*5

# your code here
def get_Ngram_distribution(N,Ngrams):
  ngram_freq = defaultdict(lambda: 0)
  ngram_dist = defaultdict(lambda: 0)
  for ngram in Ngrams:
      ngram_freq[ngram] += 1

  for ngram in ngram_freq:
      ngram_dist[ngram] = ngram_freq[ngram]/len(Ngrams)

  return ngram_dist

# now create for N = 1,2,3,4


# your code here
for N in range(1, 5):
  All_Ngram_distribution[N] = get_Ngram_distribution(N, All_Ngrams[N])

# Since all setences will start with <s>, its probability should be 1.0

All_Ngram_distribution[1][('<s>',)] = 1.0

### Part B

Now we must build a data structure to solve the following problem: If we are working in
an N-gram model for N = 2, 3, or 4, given a sequence of tokens

$$ <s>\ w_1\ w_2\ w_3\ \cdots w_i$$

generate a sample word $w_{i+1}$ using the distribution of the N-grams of the form

$$ w_{i-N+1}\ \cdots w_{i-1}\ w_{i}\  w_{i+1}.$$

In other words, we use the last $N-1$ tokens of the sequence to determine a likely
next token, given the distribution of N-grams starting with those $N-1$ tokens.

The best way to do this is to build a nested dictionary. Let us call the first $N-1$ tokens
in an N-gram the *prefix* and the last token $wn$. Then the outer dictionary
is a `defaultdict` whose keys are the prefixes and values are an inner `defaultdict`
whose keys are the $wn$ and whose values form a probability distribution for
the ways that the prefix can be completed with a token $wn$.

To give a simple example, suppose that in our corpus there are only the following bigrams whose
first token is 'the':

    ('the','boy'),    ('the','baby'),     ('the','baby'),    ('the','man')
    
Then our outer dictionary would have the prefix

    ('the',)
    
as a key, and the inner dictionary would store the probability that each of 'boy', 'baby', and 'man'
would follow 'the', as shown in the next code cell.

Note that the prefix is an N-gram (a tuple) and $wn$ is simply a token.

In [42]:
D = defaultdict(lambda: None)

D[('the',)] = defaultdict(lambda: 0)
D[('the',)]['boy'] = 0.25
D[('the',)]['man'] = 0.25
D[('the',)]['baby'] = 0.5

D

defaultdict(<function __main__.<lambda>()>,
            {('the',): defaultdict(<function __main__.<lambda>()>,
                         {'boy': 0.25, 'man': 0.25, 'baby': 0.5})})

Your task is to complete the following template and build a nested default dictionary giving
the probability distributions for completions for (N-1)-grams.

Hint:  For each N, you must create a `defaultdict` for all N-grams, whose key
is the prefix and whose values are an inner `defaultdict`. For each N-gram,
you should store the probability of the N-gram prefix+wn under the key $wn$.
Then you must normalize these probabilities so that their sum is 1.0.

The end result will be that if you look up a prefix (N-1 tokens), you will
have a probability distribution of possible $wn$ which can be used for
the next token.

In [43]:
# now build a dictionary of lists of completions, with probabilities
# For N-gram, key is prefix, of length N-1, which returns a dictionary
# with keys that are tokens (the last token wn in the N-gram), with
# values the number of times that N-gram (prefix),wn occurs.

# N here is the length of the whole N-gram, so the prefix is of length N-1

def get_Ngram_dict(N):

    ngram_dict = defaultdict(lambda: defaultdict(int))

    for ngram in All_Ngrams[N]:
        prefix = ngram[:-1]
        wn = ngram[-1]
        ngram_dict[prefix][wn] += 1

    for prefix, word in ngram_dict.items():
        total_occurrences = sum(word.values())
        for wn in word:
            ngram_dict[prefix][wn] /= total_occurrences

    return ngram_dict

Ngram_nested_dict = [None]*5

for n in range(2,5):
    Ngram_nested_dict[n] = get_Ngram_dict(n)



In [44]:
# tests: these should sum to (close to) 1.0

sum(Ngram_nested_dict[2][('<s>',)].values())

1.0000000000000744

In [45]:
sum(Ngram_nested_dict[3][('<s>','The')].values())

0.9999999999999438

In [46]:
sum(Ngram_nested_dict[4][('<s>','When', 'the')].values())

1.0000000000000007

### Part C

Now that we have a way of sampling the next likely word, we will write
a function which will predict the next word. You must sample from the
probability distribution given a prefix, to choose a likely next word.

Hint:  read about `numpy.random.choice`, in particular how you can set the parameter `p`
to determine the probability of selecting a given key from the dictionary.

In [47]:
# given a prefix, randomly choose next token using the appropriate probability distribution

#  len(prefix) must be 1, 2, 3, or 4

def next_word(prefix):

    N = len(prefix) + 1

    ngram_dict = Ngram_nested_dict[N]

    word = list(ngram_dict[prefix].keys())
    probabilities = list(ngram_dict[prefix].values())

    predicted_word = np.random.choice(word, p=probabilities)
    return predicted_word


In [48]:
# tests

next_word(('<s>',))

'You'

In [49]:
next_word(('<s>','The'))

'unhealthy'

In [50]:
next_word(('<s>','The','man'))

'moved'

### Part D

Complete the following template to generate a random sentence by starting with
the unigram `('<s>',)` and extending it by sampling until you generate the token `</s>`.

In [51]:
# N is the parameter in N-gram

def generate_sentence(N):

    sentence = ['<s>']  # Initialize the sentence with the '<s>' token

    while True:
        prefix = tuple(sentence[-(N - 1):])  # Get the current prefix based on N
        next_token = next_word(prefix)  # Sample the next token

        if next_token == '</s>':
            sentence.append(next_token)
            break  # Stop if we reach the end of the sentence
        else:
            sentence.append(next_token)  # Append the next token to the sentence

    return tuple(sentence)


In [128]:
# tests  -- run this cell many times!
w1 = generate_sentence(2)
print(np.around(PP(2,w1),2), ' '.join(w1[1:-1]),'\n')

w2 = generate_sentence(3)

print(np.around(PP(3,w2),2), ' '.join(w2[1:-1]),'\n')

w3 = generate_sentence(4)

print(np.around(PP(4,w3),2), ' '.join(w3[1:-1]),'\n')


133.14 It just been older generation to Thomas' injured his desire I was achieved ; 

5.72 From next New Year's wearing a neat , mean and bitter . 

3.38 The essential difference between the two things , or any part of the assessors are of significance to statewide planning . 



### Part E

Experiment with generating sentences for various values of N = 2, 3, 4. How do the perplexities compare?  Do you see a difference
in the quality of the sentences? How well does it do with punctuation and quotes?

The typical view is that for larger values of N,
the model is just "memorizing" the corpus. Do you think this is true? (You might look through
the corpus to see what relationship your generated sentences, say for N = 4, have with
the sentences in the corpus.

1. How do the perplexities compare?

For lower value of 'N' in the N-grams it has higher perplexities and for higher 'N' we get lower perplexities. For N = 3 & 4 we get extremely low perplexities indicating that the model is less perplexed and possibly copying words from the corpus.



2.  Do you see a difference in the quality of the sentences?

Yes, for bigrams it looks like the model is printing gibberish, mostly generating without context at all, whereas for trigrams it gets better and for quardigrams it mostly gets words from the corpus itself, so the sentences formed for quadrigrams makes most sense gramatically.

3. How well does it do with punctuation and quotes?

It seems to be inconsistent with punctuations, with random punctuations being doubled sequentially like '--' , random blank quotes, sentences ending wit ';;'.

4.The typical view is that for larger values of N, the model is just "memorizing" the corpus. Do you think this is true?

 For quadrigrams while it does memorize words from the corpus it does also generalise to some extent and so the model is not completely cheating from the corpus. On multiple attempts for different generated sentences, the block of code below has never returned true showing that the model has not directly memorised an entire sentence from the corpus.

In [129]:
for sentence in sentences[1:10]:
  if list(w3) == sentence:
    print(True)
    break

